# 📈 Fase 03: Exploratory Data Analysis (EDA)
Este notebook implementa el análisis exploratorio profundo siguiendo las reglas de la consultora Triple S para Mi Buñuelito.

### Pipeline de Análisis (7 Pasos):
1. **Temporal Splitting**: Partición determinística Train/Val/Test (Regla 3.1).
2. **Profiling & Drift**: Detección de cambios estadísticos entre periodos.
3. **Impact of Calendar**: Análisis de estacionalidad mensual y trimestral (Regla 3.3).
4. **Time Series Decomposition**: Extracción de Tendencia y Estacionalidad (Regla 3.4).
5. **ACF & PACF Analysis**: Determinación de Lags significativos (Regla 3.5).
6. **Stationarity Testing**: Validación mediante Augmented Dickey-Fuller (Regla 3.6).
7. **Business Events Validation**: Verificación de impacto de promociones y festivos (Regla 3.7).

In [ ]:
# Celda 1: Setup
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Agregar 'src' al path
sys.path.insert(0, os.path.abspath('..'))

from src.utils.config_loader import load_config
from src.explorer import DataExplorer

config = load_config("../config.yaml")

# Redirigir reportes y figuras a la carpeta de experimentos si estamos en el notebook (Lab Mode)
config['general']['paths']['reports'] = os.path.join("..", config['general']['paths']['experiments']['phase_03'], "artifacts")
config['general']['paths']['figures'] = os.path.join("..", config['general']['paths']['experiments']['phase_03'], "figures")

# Configurar el explorer para usar las rutas de laboratorio
explorer = DataExplorer(config)
# Forzar la ruta de figuras específica para la fase en el explorer
explorer.figures_path = config['general']['paths']['figures']

print(f"✅ Ambiente de EDA configurado (Modo Laboratorio).")
print(f"📂 Los reportes se guardarán en: {config['general']['paths']['reports']}")
print(f"🖼️ Las figuras se guardarán en: {config['general']['paths']['figures']}")

In [ ]:
# Celda 2: Carga del Master Cleansed Dataset
cleansed_path = os.path.join("..", config["general"]["paths"]["cleansed"], "master_cleansed.parquet")

if os.path.exists(cleansed_path):
    df = pd.read_parquet(cleansed_path)
    print(f"✅ Dataset maestro cargado: {df.shape}")
    display(df.head())
else:
    raise FileNotFoundError(f"❌ No se encontró el archivo maestro en {cleansed_path}. Ejecuta la fase de preprocesamiento primero.")

In [ ]:
# Celda 3: Ejecución del Pipeline de EDA
# El DataExplorer ejecuta automáticamente los 7 pasos definidos en el protocolo
report = explorer.run_eda(df)

print(f"\n✅ EDA completado. Status: {report['status']}")

## 📊 Hallazgos Principales

In [ ]:
# Celda 4: Resumen Estadístico y Estacionariedad
print("📊 RESULTADOS CLAVE DEL EDA:")
print(f"Status: {report['status']}")
print(f"Test de Dickey-Fuller: {report['results']['time_series']['stationarity']['verdict']}")
print(f"P-Value: {report['results']['time_series']['stationarity']['p_value']:.4f}")

print("\n🔹 IMPACTO DE EVENTOS DE NEGOCIO:")
for event, result in report['results']['business_events'].items():
    print(f"   - {event.upper()}: {result['conclusion']}")

In [ ]:
# Celda 5: Análisis de Drift (Train vs Test)
print("📑 ANALISIS DE DERIVA DE DATOS (DRIFT):")
target = config['preprocessing']['target_variable']
drift = report['results']['drift_analysis'][target]

print(f"Target: {target}")
print(f"Mean (Train): {drift['train']['mean']:.2f} vs Mean (Test): {drift['test']['mean']:.2f}")
print(f"Range (Train): [{drift['train']['min']:.2f}, {drift['train']['max']:.2f}]")
print(f"Range (Test): [{drift['test']['min']:.2f}, {drift['test']['max']:.2f}]")